In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pytorch_lightning as pl
import numpy as np
from torch.utils.data import TensorDataset, DataLoader

# Set the random seed for reproducibility
torch.manual_seed(1)

In [ ]:
# Synthetic Data Generation

def generate_synthetic_data_torch(m):
    # Generate 3 features for m examples; values in [-1, 1]
    X = torch.rand(3, m) * 2 - 1  # shape: (3, m)
    # Non-linear function: y = sin(pi*x1) + log(|x2| + 1) + x3^2
    Y = torch.sin(np.pi * X[0, :]) + torch.log(torch.abs(X[1, :]) + 1) + X[2, :]**2
    Y = Y.view(1, m)
    return X, Y

In [ ]:
# PyTorch Model using torch.nn.Module

class DeepNN(nn.Module):
    def __init__(self, layer_dims):
        super(DeepNN, self).__init__()
        layers = []
        L = len(layer_dims)
        # Create layers for l=1 to L-1: Linear + ReLU (for hidden layers)
        for l in range(1, L-1):
            layers.append(nn.Linear(layer_dims[l-1], layer_dims[l]))
            layers.append(nn.ReLU())
        # Final layer with Sigmoid activation
        layers.append(nn.Linear(layer_dims[-2], layer_dims[-1]))
        layers.append(nn.Sigmoid())
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

In [ ]:
# PyTorch Lightning Module

class LitDeepNN(pl.LightningModule):
    def __init__(self, layer_dims, learning_rate=0.01):
        super().__init__()
        self.model = DeepNN(layer_dims)
        self.learning_rate = learning_rate

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        X, Y = batch
        # At this point, X and Y are already in batch-first format: (m, features) and (m,1)
        outputs = self(X.float())
        loss = -torch.mean(Y.float()*torch.log(outputs) + (1-Y.float())*torch.log(1-outputs))
        self.log("train_loss", loss)
        return loss

    def configure_optimizers(self):
        return optim.SGD(self.parameters(), lr=self.learning_rate)

In [ ]:
# Data Preparation: Correcting the shape issue

# Generate synthetic data and transpose it BEFORE creating the dataset:
X_t, Y_t = generate_synthetic_data_torch(209)
X_t = X_t.T   # now shape: (209, 3)
Y_t = Y_t.T   # now shape: (209, 1)

# Create a TensorDataset with the transposed data
dataset = TensorDataset(X_t, Y_t)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# ----------------------------
# Train the Lightning Model
# ----------------------------
layer_dims = [3, 5, 4, 1]
lit_model = LitDeepNN(layer_dims)
trainer = pl.Trainer(max_epochs=50, logger=False, enable_checkpointing=False)
trainer.fit(lit_model, dataloader)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type   | Params | Mode 
-----------------------------------------
0 | model | DeepNN | 49     | train
-----------------------------------------
49        Trainable params
0         Non-trainable params
49        Total params
0.000     Total estimated model params size (MB)
8         Modules in train mode
0         Modules in eval mode


Training: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=50` reached.
